In [2]:
import re
import time
import numpy as np

In [6]:
import tensorflow as tf

In [8]:
lines = open('movie_lines.txt', encoding = 'utf-8',errors='ignore')
conversations = open('movie_conversations.txt', encoding = 'utf-8',errors='ignore')

In [13]:
lines = lines.read().split('\n')

In [14]:
conversations = conversations.read().split('\n')

In [20]:
id2line = {}
for line in lines:
    _line=line.split(' +++$+++ ')
    if len(_line)==5:
        id2line[_line[0]]= _line[4]

In [22]:
converstaions_ids =[]

In [53]:
for conversation in conversations[:-1]:
    _conversation = conversation.split('+++$+++')[-1][1:-1].replace("'","").replace(" ","").replace("[","")
    converstaions_ids.append(_conversation.split(','))

In [83]:
length = len(converstaions_ids)
for i in range(length):
    converstaions_ids[i][0] = converstaions_ids[i][0].replace("[","")   

In [74]:
questions=[]
answers=[]
for conversation in converstaions_ids:
    for i in range (len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [78]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"\'ll","will",text)
    return text

In [81]:
clean_questions= []
for question in questions:
    clean_questions.append(clean_text(question))

In [82]:
clean_answers= []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [84]:
word2count = {}

In [86]:
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word]= 1
        else:
            word2count[word] +=1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word]= 1
        else:
            word2count[word] +=1

In [89]:
threshold = 19

In [93]:
questionswords2int = {}
answerswords2int = {}

In [94]:
word_number = 0
for word,count in word2count.items():
    if count > threshold:
        questionswords2int[word] = word_number
        word_number += 1
word_number = 0
for word,count in word2count.items():
    if count > threshold:
        answerswords2int[word] = word_number
        word_number += 1

In [98]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [101]:
answersints2words = {w_i : 1 for w, w_i in answerswords2int.items()}

In [105]:
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'